In [ ]:
import os
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import datasets
from torch.utils.data import DataLoader
import torch
import sqlite3
from flask import Flask, request, jsonify
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#1. Pengembangan Model Pengenalan Wajah
#di saya menggunakan model FaceNet untuk pengenalan wajah. Model ini tersedia melalui facenet-pytorch.


mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

data_dir = 'data/faces'

dataset = datasets.ImageFolder(data_dir)
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}

def collate_fn(x):
    return x[0]

dataloader = DataLoader(dataset, collate_fn=collate_fn)

aligned = []
names = []

for img, idx in dataloader:
    img_cropped = mtcnn(img)
    if img_cropped is not None:
        aligned.append(img_cropped)
        names.append(dataset.idx_to_class[idx])

aligned = torch.stack(aligned)
embeddings = resnet(aligned)

np.save('data_embeddings.npy', embeddings.detach().numpy())
np.save('data_names.npy', np.array(names))

In [ ]:
#2. Perancangan Database
#saya menggunakan SQLite untuk database sederhana.

conn = sqlite3.connect('attendance.db')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS faces (
                user_id INTEGER,
                embedding BLOB NOT NULL,
                FOREIGN KEY (user_id) REFERENCES users (id)
            )''')

c.execute('''CREATE TABLE IF NOT EXISTS attendance (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INTEGER,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (user_id) REFERENCES users (id)
            )''')

conn.commit()
conn.close()

In [ ]:
#3. Integrasi dengan API
#Menggunakan Flask untuk membuat API yang akan memasukkan data kehadiran.

app = Flask(api)

def recognize_face(embedding):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    
    c.execute("SELECT user_id, embedding FROM faces")
    faces = c.fetchall()
    conn.close()
    
    min_dist = float('inf')
    recognized_user_id = None
    
    for user_id, face_embedding in faces:
        face_embedding = np.frombuffer(face_embedding, dtype=np.float32)
        dist = np.linalg.norm(face_embedding - embedding)
        if dist < min_dist:
            min_dist = dist
            recognized_user_id = user_id
    
    return recognized_user_id if min_dist < 0.6 else None

@app.route('/attendance', methods=['POST'])
def attendance():
    embedding = np.array(request.json['embedding'], dtype=np.float32)
    user_id = recognize_face(embedding)
    
    if user_id is not None:
        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()
        c.execute("INSERT INTO attendance (user_id) VALUES (?)", (user_id,))
        conn.commit()
        conn.close()
        return jsonify({"status": "success", "user_id": user_id}), 200
    else:
        return jsonify({"status": "failure"}), 400

if api == '__main__':
    app.run(debug=True)

In [ ]:
#4. DataOps Pipeline
#Mengelola aliran data untuk penambahan pengguna baru.


def add_new_user(name, img_path):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    
    c.execute("INSERT INTO users (name) VALUES (?)", (name,))
    user_id = c.lastrowid
    
    img = Image.open(img_path)
    img_cropped = mtcnn(img)
    if img_cropped is not None:
        img_embedding = resnet(img_cropped.unsqueeze(0)).detach().numpy().tobytes()
        c.execute("INSERT INTO faces (user_id, embedding) VALUES (?, ?)", (user_id, img_embedding))
    
    conn.commit()
    conn.close()


In [ ]:
#5. Visualisasi Data
#Menggunakan Matplotlib untuk visualisasi data.

embeddings = np.load('data_embeddings.npy')
names = np.load('data_names.npy')

from sklearn.decomposition import PCA

pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=embeddings_pca[:,0], y=embeddings_pca[:,1], hue=names)
plt.title('Face Embeddings PCA')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend(loc='best')
plt.show()
